# Green River Mammmoth Cave USGS Gauge Data (Site 03309000) Anlysis of flood events since removal of L&D6

In [13]:
## Installing libraries
# Install as needed.

!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install requests


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [8]:
# Import Libraries and print Versions
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import numpy as np
import os
import sys
import requests
import json

print("Python version: ", sys.version)
print("Pandas version: ", pd.__version__)
import matplotlib
print("Matplotlib version: ", matplotlib.__version__)
print("Numpy version: ", np.__version__)
print("OS version: ", os.uname())
print("OS name: ", os.name)
print("Current working directory: ", os.getcwd())
print("Python executable: ", sys.executable)
print("Python path: ", sys.path)
print("Python modules: ", sys.modules)
print("Python platform: ", sys.platform)
print("Python version: ", sys.version)
print("Python version info: ", sys.version_info)



Python version:  3.13.2 (main, Feb  4 2025, 14:51:09) [Clang 16.0.0 (clang-1600.0.26.6)]
Pandas version:  2.2.3
Matplotlib version:  3.10.1
Numpy version:  2.2.4
OS version:  posix.uname_result(sysname='Darwin', nodename='Raybiess-MacBook-Pro.local', release='24.4.0', version='Darwin Kernel Version 24.4.0: Fri Apr 11 18:33:47 PDT 2025; root:xnu-11417.101.15~117/RELEASE_ARM64_T6000', machine='arm64')
OS name:  posix
Current working directory:  /Users/raybies/CODE/GreenRiverFerryRiver/01_Scripts
Python executable:  /Users/raybies/CODE/GreenRiverFerryRiver/.venv/bin/python
Python path:  ['/opt/homebrew/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python313.zip', '/opt/homebrew/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13', '/opt/homebrew/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload', '', '/Users/raybies/CODE/GreenRiverFerryRiver/.venv/lib/python3.13/site-packages']
Python mod

In [14]:
## USGS Gauge Data
url = 'https://nwis.waterservices.usgs.gov/nwis/iv/?sites=03309000&agencyCd=USGS&startDT=2018-01-01T00:00:00.000-06:00&endDT=2025-05-19T23:59:59.999-05:00&parameterCd=00065&format=rdb'

try: 
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")
    # Handle the error appropriately (e.g., log it, retry, etc.)
    exit()

In [15]:
from io import StringIO

# Parse the response content and convert it into a DataFrame

# Decode the response content
content = response.content.decode('utf-8')

# Skip comment lines (lines starting with '#') and create a DataFrame
data_lines = "\n".join([line for line in content.splitlines() if not line.startswith('#')])
df = pd.read_csv(StringIO(data_lines), sep='\t')

# Display the first few rows of the DataFrame
print(df.head())

  agency_cd   site_no          datetime tz_cd 220645_00065 220645_00065_cd
0        5s       15s               20d    6s          14n             10s
1      USGS  03309000  2018-01-01 01:00   CST        20.15               A
2      USGS  03309000  2018-01-01 01:15   CST        20.16               A
3      USGS  03309000  2018-01-01 01:30   CST        20.15               A
4      USGS  03309000  2018-01-01 01:45   CST        20.15               A


/var/folders/yb/x_108bl52f19_7x02z9s6byc0000gn/T/ipykernel_7292/1245403309.py:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(data_lines), sep='\t')


In [16]:
# Rename columns for clarity
df = df.rename(columns={"220645_00065_cd":"approved", "tz_cd":"timezone", "220645_00065":"gage"})

# Remove the first row of the DataFrame
df = df.iloc[1:]
# Reset the index of the DataFrame
df = df.reset_index(drop=True)
# Convert the 'dateTime' column to datetime format
df['datetime'] = pd.to_datetime(df['datetime'])
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['year'] = df['datetime'].dt.year
# Create a new DataFrame with the desired columns       
df2 = df[['datetime', 'gage', 'month', 'day', 'year']]
df2.head()

,datetime,gage,month,day,year
0,2018-01-01 01:00:00,20.15,1,1,2018
1,2018-01-01 01:15:00,20.16,1,1,2018
2,2018-01-01 01:30:00,20.15,1,1,2018
3,2018-01-01 01:45:00,20.15,1,1,2018
4,2018-01-01 02:00:00,20.14,1,1,2018


/var/folders/yb/x_108bl52f19_7x02z9s6byc0000gn/T/ipykernel_7292/3519027179.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['gage'] = pd.to_numeric(df2['gage'], errors='coerce')
/var/folders/yb/x_108bl52f19_7x02z9s6byc0000gn/T/ipykernel_7292/3519027179.py:19: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(


In [18]:
# Ensure the 'gage' column in df is numeric
df['gage'] = pd.to_numeric(df['gage'], errors='coerce')

# Ensure the 'gage' column in df2 is numeric
df2['gage'] = pd.to_numeric(df2['gage'], errors='coerce')

# Create a DataFrame of the maximum gauge value for each day
daily_max_df = df.groupby(df['datetime'].dt.date)['gage'].max().reset_index()
daily_max_df.columns = ['date', 'max_daily_gage']

# Add a month column to the daily_max_df
daily_max_df['month'] = pd.to_datetime(daily_max_df['date']).dt.month

# Group the maximum daily values by month
grouped_daily_max = daily_max_df.groupby('month')['max_daily_gage']

# Create a boxplot of the maximum daily values grouped by month
plt.figure(figsize=(10, 6))
plt.boxplot(
    [group for _, group in grouped_daily_max],
    labels=grouped_daily_max.groups.keys(),
    flierprops={
        'marker': 'x',  # Outlier marker symbol
        'markerfacecolor': 'red',  # Outlier marker color
        'markeredgecolor': 'black',  # Outlier marker edge color
        'markersize': 3,  # Outlier marker size
        'alpha': 0.5  # Outlier opacity
    }
)
plt.title('Boxplot of Maximum Daily Gage Values Grouped by Month')
plt.xlabel('Month')
plt.ylabel('Maximum Daily Gage Value')
plt.grid(True)
plt.tight_layout()
plt.show()

In [19]:
# Extract data for month 4
month_2_data = daily_max_df[daily_max_df['month'] == 2]['max_daily_gage']

# Calculate Q1, Q3, and IQR
Q1 = month_2_data.quantile(0.25)
Q3 = month_2_data.quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = daily_max_df[(daily_max_df['month'] == 2) & 
                        ((daily_max_df['max_daily_gage'] < lower_bound) | 
                         (daily_max_df['max_daily_gage'] > upper_bound))]

# Output the rows for outliers
print(outliers)

            date  max_daily_gage  month
2583  2025-02-16           46.46      2
2584  2025-02-17           52.26      2
2585  2025-02-18           52.49      2
2586  2025-02-19           50.07      2


In [20]:
# Extract data for month 4
month_4_data = daily_max_df[daily_max_df['month'] == 4]['max_daily_gage']

# Calculate Q1, Q3, and IQR
Q1 = month_4_data.quantile(0.25)
Q3 = month_4_data.quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = daily_max_df[(daily_max_df['month'] == 4) & 
                        ((daily_max_df['max_daily_gage'] < lower_bound) | 
                         (daily_max_df['max_daily_gage'] > upper_bound))]

# Output the rows for outliers
print(outliers)

            date  max_daily_gage  month
2630  2025-04-04           47.73      4
2631  2025-04-05           57.67      4
2632  2025-04-06           59.54      4
2633  2025-04-07           58.99      4
2634  2025-04-08           55.82      4
2635  2025-04-09           49.47      4
2636  2025-04-10           37.50      4


In [21]:
# Extract data for month 4
month_3_data = daily_max_df[daily_max_df['month'] == 3]['max_daily_gage']

# Calculate Q1, Q3, and IQR
Q1 = month_3_data.quantile(0.25)
Q3 = month_3_data.quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = daily_max_df[(daily_max_df['month'] == 3) & 
                        ((daily_max_df['max_daily_gage'] < lower_bound) | 
                         (daily_max_df['max_daily_gage'] > upper_bound))]

# Output the rows for outliers
print(outliers)

            date  max_daily_gage  month
88    2018-03-30           33.88      3
89    2018-03-31           33.78      3
792   2020-03-03           33.17      3
793   2020-03-04           37.22      3
794   2020-03-05           37.07      3
802   2020-03-13           32.97      3
803   2020-03-14           34.50      3
804   2020-03-15           33.26      3
809   2020-03-20           34.00      3
810   2020-03-21           35.48      3
811   2020-03-22           34.99      3
1154  2021-03-01           46.12      3
1155  2021-03-02           50.76      3
1156  2021-03-03           50.94      3
1157  2021-03-04           48.02      3
1158  2021-03-05           35.80      3
2611  2025-03-16           34.08      3
2612  2025-03-17           37.96      3
2613  2025-03-18           37.67      3


In [23]:
# Extract data for month 4
month_5_data = daily_max_df[daily_max_df['month'] == 5]['max_daily_gage']

# Calculate Q1, Q3, and IQR
Q1 = month_5_data.quantile(0.25)
Q3 = month_5_data.quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = daily_max_df[(daily_max_df['month'] == 5) & 
                        ((daily_max_df['max_daily_gage'] < lower_bound) | 
                         (daily_max_df['max_daily_gage'] > upper_bound))]

# Output the rows for outliers
print(outliers)

            date  max_daily_gage  month
1219  2021-05-05           36.41      5
1220  2021-05-06           36.42      5
2300  2024-05-09           33.16      5
2301  2024-05-10           36.35      5
2302  2024-05-11           36.10      5
2318  2024-05-27           31.59      5
2319  2024-05-28           33.01      5
2673  2025-05-17           33.01      5
2674  2025-05-18           33.94      5
2675  2025-05-19           31.55      5
